# Предварительная обработка данных

## Поиск неявных дубликатов

Как упоминалось ранее, для «gender» 49 различных значений, и было подозрение, что некоторые из этих значений не следует рассматривать как разные категории. В конечном итоге для простоты мы разделим данные на 3 категории: мужчина, женщина и другие (сюда вошли те категории, которые можно однозначно исключить из предыдущих двух, для примера - трансгендер).

Если бы требовалось создать механизм предварительной обработки, который мог бы очищать входящие данные, требовалось бы воспользоваться более умным подходом. Но так как наша задача — это работа с уже имеющемся датасетом, то мы просто используем этот подход с заменой определенных типов.

```python
male_terms = ["male", "m", "mal", "msle", "malr", "mail", "make", "cis male", "man", "maile", "male (cis)", "cis man"] 
female_terms = ["female", "f", "woman", "femake", "femaile", "femake", "cis female", "cis-female/femme", "female (cis)", "femail", "cis woman"] 

def clean_gender(response): 
    if response.lower().rstrip() in male_terms: 
        return "Male" 
    elif response.lower().rstrip() in female_terms: 
        return "Female" 
    else:  
        return "Other" 

df['Gender'] = df["Gender"].apply(lambda x: clean_gender(x)) 
```

## Обработка нулевых значений

В любом наборе данных реального мира всегда есть несколько нулевых значений. На самом деле не имеет значения, является ли это регрессией, классификацией или любой другой проблемой, ни одна модель не может обрабатывать эти значения NULL или NaN самостоятельно, поэтому нам нужно вмешаться.

В питоне NULL представлен NaN. Так что не запутайтесь между этими двумя, их можно использовать взаимозаменяемо.

Прежде всего, нам нужно проверить, есть ли у нас нулевые значения в нашем наборе данных или нет. Мы можем сделать это с помощью метода isnull ().

```python
df.isnull()  
```

Есть разные способы решения этой проблемы. Самый простой способ решить эту проблему - удалить строки или столбцы, содержащие нулевые значения.

```python
df.dropna()
```

dropna () принимает различные параметры, такие как:

1. axis - мы можем указать axis = 0, если мы хотим удалить строки, и axis = 1, если мы хотим удалить столбцы.
2. how - если мы укажем how = ‘all’, тогда строки и столбцы будут отброшены только в том случае, если все значения равны NaN. По умолчанию для how задано значение any.
3. thresh - определяет пороговое значение, поэтому, если мы укажем thresh = 5, то строки, имеющие менее 5 реальных значений, будут отброшены.
4. Подмножество - если у нас есть 4 столбца A, B, C и D, то если мы укажем subset = [‘C’], то будут удалены только те строки, которые имеют значение C в виде NaN.
5. На месте - по умолчанию никакие изменения не будут внесены в ваш фрейм данных. Поэтому, если вы хотите, чтобы эти изменения отразились на вашем фрейме данных, вам нужно использовать inplace = True.

Однако это не лучший вариант удаления строк и столбцов из нашего набора данных, так как это может привести к потере ценной информации. Если у вас есть 300K точек данных, то удаление 2–3 строк не сильно повлияет на ваш набор данных, но если у вас есть только 100 точек данных, из которых 20 имеют значения NaN для определенного поля, вы не можете просто отбросить эти строки. В реальных наборах данных может случиться так, что у вас есть большое количество значений NaN для определенного поля.

Например - предположим, что мы собираем данные из опроса, тогда возможно, что может быть необязательное поле, которое, скажем, 20% людей оставит пустым. Итак, когда мы получаем набор данных, мы должны понимать, что оставшиеся 80% данных все еще полезно, поэтому вместо того, чтобы отбрасывать эти значения, нам нужно как-то заменить недостающие 20% -ые значения Мы можем сделать это с помощью **Вменение** или заменить их на отрицательное число которое не встречается в таблице например -1.



### Вменение 

Импутация - это просто процесс подстановки пропущенных значений в нашем наборе данных. Мы можем сделать это, определив нашу собственную настраиваемую функцию, или мы можем просто выполнить импутацию, используяImputerкласс предоставлен sklearn.

```python
from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values='NaN' ,strategy='mean')
imputer = imputer.fit(df[['C','D']])
df[['C','D']] = imputer.transform(df[['C','D']])
```

Здесь у нас есть два столбца с пропущенными значениями: C и D.

Значения здесь используется возвращаемое пустое представление фрейма данных.
Будут возвращены только значения во фрейме данных, метки осей будут удалены.

## Пропуски

Очень часто случается так, что в наборе данных пропущены те или иные значения. Данные с пропусками чаще всего нельзя просто так взять передать в модель. Самый простой способ избавиться от пропусков в данных — просто удалить строки, в которых есть пропущенные значения. Но бывают случаи, когда такое удаление строк может привести к потере большого количества информации.

Помимо удаления строк существуют иные, более "стойкие" к потере обучающей информации, методы. Прежде чем рассмотреть эти методы, отмечу, что признаки можно поделить на категориальные и численные.

### Пропуски в категориальных признаках

полнить пропуски в категориальных признаках можно следующими способами:
* Заменить пропущенное значение новой категорией "Неизвестно".
* Заменить пропущенное значение наиболее популярным значением.

### Пропуски в численных признаках

Если имеем дело с численными признаками, можно применить следующие подходы:
* Заменить пропущенное значение средним значением.
* Заменить пропущенное значение медианой. Если в данных присутствуют выбросы, этот способ замены пропусков является предпочтительным.

## Выбросы

В данных могут присутствовать значения, являющиеся выбросами. Это, как правило, не ошибки. Однако, своими значениями они "шокируют" модель.

Для того, чтобы определить, является ли значение выбросом, пользуются характеристикой выборки, называемой интерквартильным размахом. Определяется он следующим образом:
$$
IQR=Q_3 - Q_1
$$

где $Q_1$ — первая квартиль — такое значение признака, меньше которого ровно 25% всех значений признаков. $Q_3$ — третья квартиль — значение, меньше которого ровно 75% всех значений признака.



Для того, чтобы понять, является ли значение выбросом, можно воспользоваться эвристикой: выбросы лежат за пределами следующего интервала:

$$[ Q_1 -1.5IQR, Q_3 + 1.5IQR ]$$

Чаще всего от выбросов в обучающей выборке лучше всего избавляться.

## Нормализация

Нормализация — это приведение всех значений признака к новому диапазону. Например, к диапазону [0, 1]. Это полезно, поскольку значения признаков могут изменяться в очень большом диапазоне. Причем, значения разных признаков могут отличаться на несколько порядков. А после нормализации они все будут находиться в узком (и, часто, едином) диапазоне.

Наиболее популярным способом нормализации является нормализация методом минимакса. Для того, чтобы применить этот метод, должно быть известно максимальное и минимальное значение признака. Проблема в том, что эти значения известны не всегда.

$$ x_{new} = \frac{ x_{old} - x_{min}} {x_{max} - x_{min}} $$

Также довольно популярным методом является Z-нормализация. Диапазон новых значений для Z-нормализации выглядит следующим образом:

$$(-3σ[X], 3σ[X]),$$

где $σ[X]$ — среднеквадратическое отклонение признака X.

Выполняется Z-нормализация по формуле ниже.

$$x_{new} = \frac{x_{old} - M[X]} {σ[X]}$$

где M[X] — математическое ожидание признака X.

Отмечу, что в случае применения Z-нормализации к нескольким признакам, диапазон значений для них будет разным.

## One-hot encoding

Это способ предварительной обработки категориальных признаков. Многие модели плохо работают с категориальными признаками как таковыми. Дело в том, что слово "Российская Федерация" нельзя просто взять и умножить на какое-нибудь число. Но многие модели работают именно так: берется коэффициент и на него умножается значение признака. Аналогичная операция выполняется с остальными признаками. Все результаты суммируются. На основе значения суммы делается вывод о принадлежности объекта к тому или иному классу (такие модели называются линейными).

Однако, как поступать с признаками, значения которых нельзя выразить численно? Можно заменить их значения численным идентификатором. Например, вместо значения "Российская Федерация" использовать значение 1, а вместо "Великобритания" — 2. Тогда линейная модель будет работать. Но, если поступить таким образом, будет утеряно свойство категориальности признака. Иными словами, модель будет пытаться сравнивать идентификаторы признаков между собой. Но они не сравнимы по значению.

Чтобы бороться с этой проблемой, был придуман способ преобразовать исходный признак в несколько новых, бинарных признаков. Например, можно признак "Страна" превратить в 4 новых бинарных признака следующим образом:

| ID | Имя     | Страна Российская Федерация | Страна_Великобритания |Страна Северная Корея | Страна_Бразилия |
| -- |  ------ | --------------------------- | --------------------- | -------------------- | --------------- |
| 1  | Иван    | 1                           | 0                     | 0                    | 0               |
| 2  | Майкл   | 0                           | 1                     | 0                    | 0               |
| 3  | Ким     | 0                           | 0                     | 1                    | 0               |
| 4  | Олег    | 1                           | 0                     | 0                    | 0               |
| 5  | Педро   | 0                           | 0                     | 0                    | 1               |
| 6  | Валерий | 1                           | 0                     | 0                    | 0               |

Для каждого из этих признаков линейная модель будет использовать независимый коэффициент и, таким образом, не будет сравнивать категориальные значения одного и того же признака между собой.

One-Hot Encoding - довольно крутой и аккуратный хак, но с ним связана только одна проблема: Мультиколлинеарность. Поскольку вы все, должно быть, предположили, что это довольно тяжелое слово, поэтому его должно быть трудно понять, поэтому позвольте мне подтвердить ваше вновь сформировавшееся убеждение. Мультиколлинеарность - это действительно немного хитрая, но чрезвычайно важная концепция статистики. Здесь хорошо то, что нам не нужно понимать все мельчайшие детали мультиколлинеарности, нам просто нужно сосредоточиться на том, как это повлияет на нашу модель. Итак, давайте углубимся в эту концепцию мультиколлинеарности и как она повлияет на нашу модель.

### Мультиколлинеарность

Мультиколлинеарность возникает в нашем наборе данных, когда у нас есть особенности, которые сильно зависят друг от друга. В этом случае у нас есть особенности -

color_blue, color_green и color_white, которые все зависят друг от друга, и это может повлиять на нашу модель.

* Основное влияние это окажет на то, что это может вызвать Граница решения изменить что может оказать огромное влияние на результат нашей модели.
* В дополнение к этому, если у нас есть мультиколлинеарность в нашем наборе данных, мы не сможем использовать наш весовой вектор для вычисления важности функции.

Чтобы избежать Multicollinearity

Мы можем использовать drop_first = True, чтобы избежать проблемы мультиколлинеарности.

```python
df_cat = pd.get_dummies(df_cat[['color','size','price']],drop_first=True)
```


Здесь drop_first удалит первый столбец цвета. Поэтому здесь color_blue будет удален, и у нас будут только color_green и color_white.

Здесь важно отметить, что мы не теряем никакой информации, потому что если color_green и color_white равны 0, то это означает, что цвет должен быть синим. Таким образом, мы можем вывести всю информацию с помощью только этих 2 столбцов, следовательно, сильная взаимосвязь между этими тремя столбцами нарушена.

## Разбор структурированных данных из JSON

Попробуем «разобрать» json-подобный объект.
Для этого воспользуемся pandas.json_normalize, который разбирает структурированные данные из JSON в табличный формат, а также json.load, который десериализует текст или байткод, содержащий json-документ в python-объекты. Чтобы применить эти методы ко всей серии, воспользуемся DataFrame.apply (метод не сильно быстрый, но нам требуется выполнить эту операцию всего один раз, поэтому, в данном случае, время смело приносим в жертву).

```python
df_train_extracted = pd.io.json.json_normalize(
    df_train['event_data'].apply(json.loads))
```

Проблема заключается в том, что не все json-подобные объекты удалось полностью разобрать, т.к. на самом деле они не отвечают стандарту json. Например, вот эти колонки остались неразобранными:

lower	flowers
0.0	[0, 0, 0, 0, 0]
0.0	[0, 0, 0, 0, 8]
0.0	[8, 8, 8, 7, 8]
shels
[2, 3, 1]
[2, 3, 2, 1]
[2, 3, 2, 3, 2]

Мы видим списки с пропусками, пустые списки, списки неравной длины, списки словарей, а так-же списки с разным числом вложенных вловарей и даже кортежи списков. Все это возникло потому, что так когда-то кому-то было удобно складировать эти данные в базу данных. Теперь это придется разобрать вручную.

## Кодирование данных

На самом деле Pandas группирует и хранит “столбцы” блоками, разбитыми по типам. Иными словами float, int и objects хранятся раздельно, причем оптимизировано, без индексов. С числами все просто — столбцы в блоке объединяются в многомерный массив NumPy. При запросе значения происходит сопоставление индекса с массивом. С объектами немного сложнее. Все это означает, что разные объекты по-разному используют память.

```python
numerics_part.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 66 columns):
event_code                  200000 non-null int64
event_count                 200000 non-null int64
game_time                   196724 non-null float64
total_duration              34617 non-null float64
duration                    67500 non-null float64
...
end_position                4 non-null float64
gate.row                    832 non-null float64
gate.column                 832 non-null float64
dinosaur_weight             1547 non-null float64
dinosaur_count              1550 non-null float64
dtypes: float64(64), int64(2)
memory usage: 100.7 MB
```

Изменим эту ситуацию и вот как: получим минимальное и максимальное значение в серии, затем сравним его с машинными лимитами для типов Numpy, после чего заменим на наименьшее.

```python
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
for col in df.columns:
    col_type = df[col].dtypes
    if col_type in numerics:
        c_min = df[col].min()
        c_max = df[col].max()
        if str(col_type)[:3] == 'int':
            # последовательно сравниваем от наименьшего инта начиная с np.int8
            # наверх и переопределяем тип для серии
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            # и т.д.
        else:
            # аналогично для float
```

Мы выполнили самопальный вариант понижающего преобразования, выиграв 74.4Mb.

```python
numerics_part.info(memory_usage='deep', max_cols=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 66 entries, event_code to dinosaur_count
dtypes: float16(61), float32(3), int16(2)
memory usage: 26.3 MB
```

Но у нас все еще есть проблема:

Оказывается, везде в датасете использовано число с плавающей точкой, хотя на самом деле никакой потребности в этом нет - в этих данных все числа целые. Исправим эту ситуацию:

```python
numerics_part.fillna(value=-1, inplace=True)
numerics_part = numerics_part.astype(int)
```

**Переходим к объектам.** Тип object в Pandas хранит строковое представление. Строки хранятся фрагментировано - значение в ячейке по сути является указателем. При этом резервируется много памяти и это для нас плохо.

Pandas предоставляет подтип category, который отображает строковые данные на индекс в int, а это то, что нам нужно, т.к. данные будут храниться не в виде указателя, а в виде словаря, в котором целочисленным значениям сопоставлены уникальным значениям данных. Перегоним наши объекты в «категории»:

```python
# subtype categoty
def object_to_category(part):
    converted_objects_part = pd.DataFrame()
    unconverted_objects_part = pd.DataFrame()
    total = len(part)
    for col in part.columns:
        try:
            unic = len(part[col].unique())
            if (unic / total) < 0.05:
                converted_objects_part.loc[:,col] = part[col].astype('category')
            else:
                converted_objects_part.loc[:,col] = part[col]
        except TypeError:
            # unhashable objects can't be categorised
            unconverted_objects_part.loc[:,col] = part[col]
    return converted_objects_part, unconverted_objects_part
```

Мы отбросим часть объектов, т.к. договорились ранее не работать с нераспакованной частью json

```python
converted_objects_part, unconverted_objects_part = object_to_category(objects_part)
converted_objects_part.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 39 columns):
version                2345 non-null category
description            69717 non-null category
identifier             68610 non-null category
...
crystal_id             2090 non-null category
location               1180 non-null category
gate.side              832 non-null category
dtypes: category(39)
memory usage: 7.9 MB
```

Временную метку оптимизируем с помощью функции pandas pd.to_datetime. Параметр format позволяет задать тип представления временной отметки. Дефолтный вот такой: “%d/%m/%Y”. Теперь метка будет выглядеть так:

```python
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
df_train['timestamp']

0        2019-09-06 17:53:46.937000+00:00
1        2019-09-06 17:54:17.519000+00:00
2        2019-09-06 17:54:56.302000+00:00
...
199997   2019-08-02 00:06:37.107000+00:00
199998   2019-08-02 00:06:38.480000+00:00
199999   2019-08-02 00:06:40.684000+00:00
Name: timestamp, Length: 200000, dtype: datetime64[ns, UTC]
```